In [15]:
#imports for reading other files
import io
import os
from nbformat import read
import sys
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
###################################################

#imports for the main menu
import ipywidgets as widgets
from IPython.display import display, clear_output

original_dir = specified_dir = "../All_tests_launcher/"

class TestsRunner:
    def __init__(self, home_screen):
        self.home_screen = home_screen
        
    def run_notebook_code(self, notebook_path):
        # Define the lexer and formatter
        lexer = PythonLexer()
        formatter = HtmlFormatter()

        # Read the notebook
        with io.open(notebook_path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)

        # Get the global namespace
        global_namespace = globals()

        # Execute the code cells in the global scope
        for cell in nb.cells:
            if cell.cell_type == 'code':
                # Change directory before executing each cell
                os.chdir(f"../{self.home_screen.current_test}")
                
                # Execute cell source in the global namespace
                exec(cell.source, global_namespace)

        # Change back to the original directory after all cells have been executed
        os.chdir(original_dir)

class HomeScreen:
    def __init__(self):
        self.tests_runner = TestsRunner(self)
        self.tracking_taken_tests = {"Ans_Test": 0, "Math_Test": 0, "Memory_Test": 0, "Spatial_Test": 0}
        self.output = widgets.Output()
        
    def setup_test_interaction(self, test_name):
        """Prepares the interaction logic for each test button."""
        def handle_test_execution(button_instance):
            """Executes the selected test and updates the button upon completion."""
            self.current_test = test_name
            with self.output:
                clear_output()  # Clear the output of the current cell
                script_path = f"../{self.current_test}/{self.current_test}.ipynb"
                self.tests_runner.run_notebook_code(script_path)

                # Update the test's status and button appearance if it's the first completion
                if self.tracking_taken_tests[self.current_test] == 0:
                    button_instance.description = f"{test_name} ✔️"
                    button_instance.style.button_color = 'lightgreen'
                    self.tracking_taken_tests[self.current_test] = 1

        return handle_test_execution
        
    def home_screen_widgets(self):
        home_text = widgets.HTML(
            value="<div style='text-align: center; font-size: 16px;'>"
            "<h2>Welcome to the UCL BIOS0030 cognitive tasks main menu</h2>"
            "<p>You will have a chance to complete four different cognitive tasks, each testing different abilities.<br>"
            "You may wish to allocate 10-15 minutes to complete all of them. After completing each task, the widgets will be marked,"
            "to help you keep track of your progress.<br>Most importantly, <u>take them with a grain of salt</u>. Just like humans, cognitive tests are not perfect.<br>"
            "<b>Have fun!</b></div>",  # Added closing </div>
            layout=widgets.Layout(justify_content='center')
        )
        display(home_text)
    
        choices_dictionary = {
            
            "ANS_Test": {
                "photo": widgets.Image(value=open("ans.png", "rb").read(), format='png', width=150, height=150),
                "button": widgets.Button(description="ANS Test")
            },
            "Maths_Test": {
                "photo": widgets.Image(value=open("math_photo.png", "rb").read(), format='png', width=150, height=150),
                "button": widgets.Button(description="Maths Test")
            },
            "Memory_Test": {
                "photo": widgets.Image(value=open("memory.png", "rb").read(), format='png', width=150, height=150),
                "button": widgets.Button(description="Memory Test")
            },
            "Spatial_Test": {
                "photo": widgets.Image(value=open("puzzle.png", "rb").read(), format='png', width=150, height=150),
                "button": widgets.Button(description="Spatial Test")
            }
        }
    
        for test, widget in choices_dictionary.items():
            widget["button"].on_click(self.setup_test_interaction(test))  # Corrected the parenthesis
    
        # Grid layout
        grid = widgets.GridspecLayout(2, 4)  # Removed the colon
        for i, (test, test_widgets) in enumerate(choices_dictionary.items()):  # Renamed 'widgets' to 'test_widgets'
            grid[0, i] = test_widgets["photo"]
            grid[1, i] = test_widgets["button"]

        vboxed_grid = widgets.VBox([grid],
                            layout=widgets.Layout(align_items='center', margin='100px 0 0 0'))
        
        display(vboxed_grid)
    
#Create an instance of home_screen
os.chdir(specified_dir)
home = HomeScreen()
home.home_screen_widgets()
display(home.output) 

HTML(value="<div style='text-align: center; font-size: 16px;'><h2>Welcome to the UCL BIOS0030 cognitive tasks …

Output()